In [1]:
import pydfs_lineup_optimizer
import pulp
import numpy as np
import pandas as pd
import openpyxl
import os
import sys





In [7]:

from pydfs_lineup_optimizer import get_optimizer, Site, Sport


optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)



optimizer.load_players_from_csv("../SD_Proj.csv")











IndexError: list index out of range

Create our variables first

   Position                   Name + ID             Name        ID  \
5        QB  Patrick Mahomes (30190044)  Patrick Mahomes  30190044   
7        TE     Travis Kelce (30190045)     Travis Kelce  30190045   
10       QB   Russell Wilson (30190046)   Russell Wilson  30190046   
12       RB    Isiah Pacheco (30190047)    Isiah Pacheco  30190047   
13       WR      Jerry Jeudy (30190048)      Jerry Jeudy  30190048   
..      ...                         ...              ...       ...   
82       WR    David Sills V (30190039)    David Sills V  30190039   
83       WR     Daniel Arias (30190040)     Daniel Arias  30190040   
84       TE      Lucas Krull (30190041)      Lucas Krull  30190041   
85       TE     Jody Fortson (30190042)     Jody Fortson  30190042   
86       WR   Tre'Quan Smith (30190043)   Tre'Quan Smith  30190043   

   Roster Position  Salary                     Game Info TeamAbbrev  \
5             FLEX   12800  DEN@KC 10/12/2023 08:15PM ET         KC   
7             FLE

In [107]:
solver = pulp.PULP_CBC_CMD(msg=False)
problem = pulp.LpProblem("Problem", pulp.LpMaximize)
players = pd.read_excel('../NFL_SD_Proj_Week6.xlsx')

bin_var=[]
proj_list=[]
for i in range(len(players)):
    if players.loc[i,'Roster Position']=='CPT':
        bin_var.append(pulp.LpVariable("CPT_"+str(players.loc[i,'Name + ID']), cat='Binary'))
        proj_list.append(1.5*players.loc[i,'FLEX Proj'])
    else:
        bin_var.append(pulp.LpVariable("FLEX_"+str(players.loc[i,'Name + ID']), cat='Binary'))
        proj_list.append(players.loc[i,'FLEX Proj'])

players['bin_var']=bin_var
players['proj']=proj_list

#Sort DF to make things easier in the long run
players = players.sort_values(by=['Roster Position','proj'],ascending=False)
print(players)

problem += pulp.lpDot(players['bin_var'],players['Salary']) <= 50000
problem += pulp.lpDot([1]*len(list(filter(lambda x:x.name[0:3]=='FLE',players['bin_var']))),list(filter(lambda x:x.name[0:3]=='FLE',players['bin_var']))) == 5
problem += pulp.lpDot([1]*len(list(filter(lambda x:x.name[0:3]=='CPT',players['bin_var']))),list(filter(lambda x:x.name[0:3]=='CPT',players['bin_var']))) == 1
for i in range(len(list(filter(lambda x:x.name[0:3]=='CPT',players['bin_var'])))):
    problem += list(filter(lambda x:x.name[0:3]=='CPT',players['bin_var']))[i] + list(filter(lambda x:x.name[0:3]=='FLE',players['bin_var']))[i] <= 1
problem += pulp.lpDot(players['bin_var'],players['proj'])


N=50
scores=[]

for i in range(N):
    problem.solve()
    score=pulp.value(problem.objective)
    scores.append(score)
    print('score: '+str(score))
    problem += problem.objective <= score - 1e-3



scores

   Position                   Name + ID             Name        ID  \
5        QB  Patrick Mahomes (30190044)  Patrick Mahomes  30190044   
7        TE     Travis Kelce (30190045)     Travis Kelce  30190045   
10       QB   Russell Wilson (30190046)   Russell Wilson  30190046   
12       RB    Isiah Pacheco (30190047)    Isiah Pacheco  30190047   
13       WR      Jerry Jeudy (30190048)      Jerry Jeudy  30190048   
..      ...                         ...              ...       ...   
82       WR    David Sills V (30190039)    David Sills V  30190039   
83       WR     Daniel Arias (30190040)     Daniel Arias  30190040   
84       TE      Lucas Krull (30190041)      Lucas Krull  30190041   
85       TE     Jody Fortson (30190042)     Jody Fortson  30190042   
86       WR   Tre'Quan Smith (30190043)   Tre'Quan Smith  30190043   

   Roster Position  Salary                     Game Info TeamAbbrev  \
5             FLEX   12800  DEN@KC 10/12/2023 08:15PM ET         KC   
7             FLE

[92.52600000000001,
 92.498,
 92.22200000000001,
 92.14600000000002,
 92.11800000000001,
 91.88600000000001,
 91.637,
 91.56400000000001,
 91.356,
 91.328,
 91.27800000000002,
 91.11000000000001,
 91.105,
 91.063,
 91.054,
 91.03800000000001,
 90.976,
 90.96000000000001,
 90.94800000000001,
 90.906,
 90.86800000000001,
 90.762,
 90.73400000000001,
 90.71000000000001,
 90.686,
 90.65800000000002,
 90.644,
 90.61600000000001,
 90.587,
 90.559,
 90.48400000000001,
 90.4,
 90.37200000000001,
 90.328,
 90.31,
 90.30000000000001,
 90.268,
 90.24000000000001,
 90.23600000000002,
 90.177,
 90.17600000000002,
 90.16,
 90.149,
 90.14800000000001,
 90.132,
 90.108,
 90.06200000000001,
 90.05000000000001,
 90.048,
 90.02800000000002]

In [98]:
problem.objective

7.470000000000001*CPT_Adam_Trautman_(30190011) + 0.8370000000000001*CPT_Blake_Bell_(30190019) + 5.955*CPT_Brandon_Johnson_(30190013) + 0.8940000000000001*CPT_Chris_Manhertz_(30190032) + 5.913*CPT_Clyde_Edwards_Helaire_(30190014) + 16.455000000000002*CPT_Courtland_Sutton_(30189992) + 1.737*CPT_Greg_Dulcich_(30190027) + 13.950000000000001*CPT_Harrison_Butker_(30190004) + 23.625*CPT_Isiah_Pacheco_(30189990) + 7.41*CPT_Jaleel_McLaughlin_(30189994) + 14.739*CPT_Javonte_Williams_(30189993) + 8.693999999999999*CPT_Jerick_McKinnon_(30190002) + 18.129000000000005*CPT_Jerry_Jeudy_(30189991) + 7.38*CPT_Justin_Watson_(30190012) + 3.7230000000000003*CPT_Justyn_Ross_(30190015) + 10.767000000000001*CPT_Kadarius_Toney_(30190008) + 1.881*CPT_Lil'Jordan_Humphrey_(30190028) + 7.38*CPT_Marquez_Valdes_Scantling_(30190009) + 7.7669999999999995*CPT_Marvin_Mims_Jr._(30190005) + 0.6000000000000001*CPT_Michael_Burton_(30190026) + 1.116*CPT_Nate_Adkins_(30190033) + 6.618*CPT_Noah_Gray_(30190006) + 35.93400000000

In [ ]:
print(dir(bin_var[0]))
list(filter(lambda x: x.name[:3]=='CPT',bin_var))








['_LpElement__name', '__add__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__radd__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_lowbound_original', '_upbound_original', 'addVariableToConstraints', 'add_expression', 'asCplexLpAffineExpression', 'asCplexLpVariable', 'bounds', 'cat', 'dict', 'dicts', 'dj', 'expression', 'fixValue', 'fromDict', 'from_dict', 'getLb', 'getName', 'getUb', 'hash', 'illegal_chars', 'infeasibilityGap', 'isBinary', 'isConstant', 'isFixed', 'isFree', 'isInteger', 'isPositive', 'lowBound', 'matrix', 'modified', 'name', 'positive', 'round', 'roundedValue', 'setInitialValue', 'setName', 'toDic

/opt/homebrew/lib/python3.11/site-packages/pulp/pulp.py:1537: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Problem:
MAXIMIZE
7.470000000000001*CPT_Adam_Trautman_(30190011) + 0.8370000000000001*CPT_Blake_Bell_(30190019) + 5.955*CPT_Brandon_Johnson_(30190013) + 0.8940000000000001*CPT_Chris_Manhertz_(30190032) + 5.913*CPT_Clyde_Edwards_Helaire_(30190014) + 16.455000000000002*CPT_Courtland_Sutton_(30189992) + 1.737*CPT_Greg_Dulcich_(30190027) + 13.950000000000001*CPT_Harrison_Butker_(30190004) + 23.625*CPT_Isiah_Pacheco_(30189990) + 7.41*CPT_Jaleel_McLaughlin_(30189994) + 14.739*CPT_Javonte_Williams_(30189993) + 8.693999999999999*CPT_Jerick_McKinnon_(30190002) + 18.129000000000005*CPT_Jerry_Jeudy_(30189991) + 7.38*CPT_Justin_Watson_(30190012) + 3.7230000000000003*CPT_Justyn_Ross_(30190015) + 10.767000000000001*CPT_Kadarius_Toney_(30190008) + 1.881*CPT_Lil'Jordan_Humphrey_(30190028) + 7.38*CPT_Marquez_Valdes_Scantling_(30190009) + 7.7669999999999995*CPT_Marvin_Mims_Jr._(30190005) + 0.6000000000000001*CPT_Michael_Burton_(30190026) + 1.116*CPT_Nate_Adkins_(30190033) + 6.618*CPT_Noah_Gray_(30190006

In [ ]:
pulp.lpDot([1]*3,[1,2,3])

6